In [3]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_39114/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [1]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [2]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_38_9_24,0.999962,0.770471,0.999977,0.999626,0.999909,0.000054,0.325783,0.000063,0.000252,0.000158,0.006553,0.007377,1.000001,0.007691,1689.637512,2707.398826,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
1,model_38_9_23,0.999962,0.770487,0.999977,0.999632,0.999911,0.000055,0.325760,0.000063,0.000248,0.000156,0.006592,0.007392,1.000001,0.007707,1689.629356,2707.390670,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
2,model_38_9_22,0.999961,0.770505,0.999977,0.999639,0.999912,0.000055,0.325735,0.000063,0.000243,0.000153,0.006632,0.007418,1.000001,0.007734,1689.615485,2707.376799,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
3,model_38_9_21,0.999961,0.770524,0.999977,0.999647,0.999913,0.000056,0.325707,0.000064,0.000238,0.000151,0.006682,0.007457,1.000001,0.007775,1689.594315,2707.355629,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
4,model_38_9_20,0.999960,0.770546,0.999977,0.999656,0.999915,0.000056,0.325677,0.000064,0.000232,0.000148,0.006735,0.007514,1.000001,0.007834,1689.563863,2707.325176,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
5,model_38_9_19,0.999959,0.770569,0.999977,0.999665,0.999917,0.000058,0.325643,0.000064,0.000226,0.000145,0.006794,0.007593,1.000001,0.007917,1689.521981,2707.283295,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
6,model_38_9_18,0.999958,0.770596,0.999977,0.999675,0.999918,0.000059,0.325606,0.000065,0.000219,0.000142,0.006858,0.007700,1.000001,0.008028,1689.465951,2707.227265,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
7,model_38_9_17,0.999957,0.770624,0.999976,0.999687,0.999920,0.000061,0.325565,0.000066,0.000211,0.000138,0.006933,0.007842,1.000001,0.008176,1689.393161,2707.154475,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
8,model_38_9_16,0.999955,0.770656,0.999976,0.999699,0.999922,0.000064,0.325520,0.000067,0.000203,0.000135,0.007016,0.008025,1.000001,0.008367,1689.300585,2707.061899,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
9,model_38_9_15,0.999952,0.770691,0.999975,0.999712,0.999924,0.000068,0.325471,0.000069,0.000194,0.000131,0.007104,0.008260,1.000001,0.008611,1689.185547,2706.946861,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
